## Database Connection

Connecting to the database:

In [1]:
from data import DataManager

imdb_data = DataManager(
    dbname='imdb',
    user='samaher',
    password="CodingIsFun++",
    host='localhost',
    port='5432'
)

Creating 5 dataframes for the tables I extracted from the database: 
- movie
- genre
- prod
- rating
- crew

In [2]:
# Get data from the movie table
movie_data = imdb_data.get_movie()

# Get data from the genre table
genre_data = imdb_data.get_genre()

# Get data from the produced table
prod_data = imdb_data.get_prod()

# Get data from the rating table
rating_data = imdb_data.get_rating()

# Get data from the crew table
crew_data = imdb_data.get_crew()


I will define the levels of difficulty of the questions based on the popularity of the movies. 

- easy: 500'000+ votes
- medium: 100'000 <= votes < 500'000
- hard: votes < 100'000 

In [ ]:
import pandas as pd

# Creating a dictionary for the difficulty
difficulty = {
    'easy': [],
    'medium': [],
    'hard': []
}

# Categorizing the difficulty level of all movies according to their popularity
for index, row in rating_data.iterrows():
    try:
        votes = int(row['votes'])
        category = 'easy' if votes >= 500000 else ('medium' if 100000 <= votes < 500000 else 'hard')
        difficulty[category].append(row['official_title'])
    except ValueError:
        print(f"Invalid votes value for '{row['official_title']}' at index {index}") # Just making sure all movies are categorized

# Printing the result
for level, titles in difficulty.items():
    print(f"{level.capitalize()} Movies:", titles)



I want to check the level of difficulty of a particular movie.

In [ ]:
# Creating the function that gets the level of difficulty of a particular movie
def get_difficulty(movie_title):
    for level, titles in difficulty.items():
        if movie_title in titles:
            return level
    return None 

# Checking if the function works with the example of 'Interstellar'
movie_to_check = 'Interstellar'
difficulty_level = get_difficulty(movie_to_check)

if difficulty_level:
    print(f"The difficulty level of '{movie_to_check}' is: {difficulty_level.capitalize()}")
else:
    print(f"'{movie_to_check}' not found in any difficulty level.") # This is to raise if there's an error

I want to create a question that has the difficulty level 'easy'

- questions: list of questions from the table movie
- correct answer: the cell in the corresponding row of the column 'year' in the movie table
- options: 3 random options from random cells in the column 'year' in the movie table BUT different from the correct answer
- level: choose the level of difficulty

In [ ]:
movie_data.columns

In [ ]:
# Creating all possible questions from the movie table 
import random

for index, row in movie_data.iterrows():
    official_title = row['official_title']
    movie_question = f'When was the movie {official_title} released?'
    movie_correct_answer = row['year']

    # Getting all unique years excluding the correct answer
    all_years = list(set(movie_data['year'].unique()))
    all_years.remove(movie_correct_answer)  # Removing the correct answer

    # Selecting 3 more random incorrect options
    movie_options = [movie_correct_answer] + random.sample(all_years, 3)

    # Shuffling the options to randomize their order
    random.shuffle(movie_options)

    print(movie_question)
    print("Options:", movie_options)
    print("Correct Answer:", movie_correct_answer)
    print()




In [ ]:
import random

# Creating a random movie question with a random level of difficulty along with options and correct answer
def generate_movie_question(row, difficulty_level):
    official_title = row['official_title']
    movie_correct_answer = row['year']

    # Getting all unique years excluding the correct answer
    all_years = list(set(movie_data['year'].unique()))
    all_years.remove(movie_correct_answer)  # Removing the correct answer

    # Selecting 3 more random incorrect options
    movie_options = [movie_correct_answer] + random.sample(all_years, 3)

    # Shuffling the options to randomize their order
    random.shuffle(movie_options)

    # Constructing the question dictionary
    question_dict = {
        'question': f'When was the movie {official_title} released?',
        'options': movie_options,
        'correct_answer': movie_correct_answer,
        'difficulty_level': difficulty_level
    }

    return question_dict

difficulty_levels = ['easy', 'medium', 'hard']

# Example usage for a movie with user-specified difficulty level
row_index = random.randint(0, len(movie_data) - 1)
user_difficulty = random.choice(difficulty_levels)
question_info = generate_movie_question(movie_data.iloc[row_index], difficulty_level=user_difficulty)

# Print the question information
print(question_info['question'])
print("Options:", question_info['options'])
print("Correct Answer:", question_info['correct_answer'])
print("Difficulty Level:", question_info['difficulty_level'])



I want to nake the question interative, and make all options start with a letter from A, B, C, D.

In [ ]:
import random

def generate_movie_question(row, difficulty_level):
    official_title = row['official_title']
    movie_correct_answer = row['year']

    # Getting all unique years excluding the correct answer
    all_years = list(set(movie_data['year'].unique()))
    all_years.remove(movie_correct_answer)  # Removing the correct answer

    # Selecting 3 more random incorrect options
    movie_options = [movie_correct_answer] + random.sample(all_years, 3)

    # Shuffling the options to randomize their order
    random.shuffle(movie_options)

    # Mapping options to letters (A, B, C, D)
    options_mapping = {chr(ord('A') + i): option for i, option in enumerate(movie_options)}

    # Constructing the question dictionary
    question_dict = {
        'question': f'When was the movie {official_title} released?',
        'options': options_mapping,
        'correct_answer': chr(ord('A') + movie_options.index(movie_correct_answer)),
        'difficulty_level': difficulty_level
    }

    return question_dict

def quiz_game():
    difficulty_levels = ['easy', 'medium', 'hard']

    # Getting user input for difficulty level
    user_difficulty = input("Choose a difficulty level (easy, medium, hard): ").lower()

    # Validating user input
    while user_difficulty not in difficulty_levels:
        print("Invalid difficulty level. Please choose from: easy, medium, hard")
        user_difficulty = input("Choose a difficulty level (easy, medium, hard): ").lower()

    # Generating a random row index
    row_index = random.randint(0, len(movie_data) - 1)

    # Getting the corresponding row from the DataFrame
    row = movie_data.iloc[row_index]

    # Generating the question
    question_info = generate_movie_question(row, difficulty_level=user_difficulty)

    # Print the question information
    print(question_info['question'])
    
    # Print options with letters (A, B, C, D)
    for letter, option in question_info['options'].items():
        print(f"{letter}. {option}")

    # Get user's choice
    user_choice = input("Enter your choice (A, B, C, D): ").upper()

    # Check if the user's choice is correct
    if user_choice == question_info['correct_answer']:
        print("Correct!")
    else:
        print(f"Wrong! The correct answer is: {question_info['correct_answer']}")

# Call the quiz_game function to play the game
quiz_game()


Now, I want to add the score. 
It is calculated this way: 
wrong answer: 0 points
correct answer:
- 1 if EASY
- 2 if MEDIUM
- 3 if HARD

In [ ]:
import random

def generate_movie_question(row, difficulty_level):
    official_title = row['official_title']
    movie_correct_answer = row['year']

    # Getting all unique years excluding the correct answer
    all_years = list(set(movie_data['year'].unique()))
    all_years.remove(movie_correct_answer)  # Removing the correct answer

    # Selecting 3 more random incorrect options
    movie_options = [movie_correct_answer] + random.sample(all_years, 3)

    # Shuffling the options to randomizing their order
    random.shuffle(movie_options)

    # Mapping options to letters (A, B, C, D)
    options_mapping = {chr(ord('A') + i): option for i, option in enumerate(movie_options)}

    # Constructing the question dictionary
    question_dict = {
        'question': f'When was the movie {official_title} released?',
        'options': options_mapping,
        'correct_answer': chr(ord('A') + movie_options.index(movie_correct_answer)),
        'difficulty_level': difficulty_level
    }

    return question_dict

def calculate_score(difficulty_level, is_correct):
    if is_correct:
        if difficulty_level == 'easy':
            return 1
        elif difficulty_level == 'medium':
            return 2
        elif difficulty_level == 'hard':
            return 3
    else:
        return 0

def quiz_game():
    difficulty_levels = ['easy', 'medium', 'hard']

    # Getting user input for difficulty level
    user_difficulty = input("Choosing a difficulty level (easy, medium, hard): ").lower()

    # Validating user input
    while user_difficulty not in difficulty_levels:
        print("Invalid difficulty level. Please choose from: easy, medium, hard")
        user_difficulty = input("Choosing a difficulty level (easy, medium, hard): ").lower()

    # Generating a random row index
    row_index = random.randint(0, len(movie_data) - 1)

    # Getting the corresponding row from the DataFrame
    row = movie_data.iloc[row_index]

    # Generating the question
    question_info = generate_movie_question(row, difficulty_level=user_difficulty)

    # Printing the question information
    print(question_info['question'])
    
    # Printing options with letters (A, B, C, D)
    for letter, option in question_info['options'].items():
        print(f"{letter}. {option}")

    # Getting user's choice
    user_choice = input("Entering your choice (A, B, C, D): ").upper()

    # Checking if the user's choice is correct
    is_correct = user_choice == question_info['correct_answer']

    # Calculating and displaying the score
    score = calculate_score(question_info['difficulty_level'], is_correct)
    
    # Providing feedback on the answer
    if is_correct:
        print("Correct!")
    else:
        print(f"Wrong! The correct answer is: {question_info['correct_answer']}")

    print(f"Your score: {score}")

# Calling the quiz_game function to play the game
quiz_game()


### Difficulty Level

Now, I want to add another layer of difficulty based on the recency of the release where the levels are: 

easy: if the movie was released starting from 2010
medium: if the movie was released between 1990 and 2010
hard: if the movie was released before 1990

which makes the overall level of difficulty as follows

easy: if the movie was released starting from 2010 and has more than 500000 votes
medium: if the movie was released between 1990 and 2010 and has between 100000 abd 500000
hard: if the movie was released before 1990 and has less than 100000 votes

In [3]:
def determine_difficulty_level(year, votes):
    if (year >= 2010 and votes >= 500000) or (year < 2010 and votes >= 1000000):
        return 'easy'
    elif 1990 <= year < 2010 and 100000 <= votes < 500000:
        return 'medium'
    elif year < 1990 and votes < 100000:
        return 'hard'
    else:
        return 'unknown'  # to check for errors


In [4]:
import pandas as pd

# Creating a dictionary for the difficulty
difficulty = {
    'easy': [],
    'medium': [],
    'hard': []
}

# Categorizing the difficulty level of all movies according to their popularity and release year
for index, row in rating_data.iterrows():
    try:
        votes = int(row['votes'])
        year = int(row['year'])
        
        difficulty_level = determine_difficulty_level(year, votes)

        difficulty[difficulty_level].append(row['official_title'])
    except (ValueError, KeyError):
        print(f"Undetermined level of difficulty for'{row.get('official_title', 'Unknown')}' at index {index}")

# Printing the result
for level, titles in difficulty.items():
    print(f"{level.capitalize()} Movies:", titles)


Undetermined level of difficulty for'The Birds' at index 4
Undetermined level of difficulty for'Psycho' at index 7
Undetermined level of difficulty for'North by Northwest' at index 8
Undetermined level of difficulty for'Vertigo' at index 10
Undetermined level of difficulty for'Rear Window' at index 15
Undetermined level of difficulty for'Dial M for Murder' at index 16
Undetermined level of difficulty for'Strangers on a Train' at index 18
Undetermined level of difficulty for'Rope' at index 21
Undetermined level of difficulty for'Memory of the Camps' at index 25
Undetermined level of difficulty for'Don't Give Me the Finger' at index 66
Undetermined level of difficulty for'Citizen Kane' at index 86
Undetermined level of difficulty for'Moby Dick' at index 89
Undetermined level of difficulty for'It's All True' at index 91
Undetermined level of difficulty for'Don Quijote de Orson Welles' at index 92
Undetermined level of difficulty for'Citizen Vader' at index 110
Undetermined level of diffic

In [5]:
import random

def generate_movie_question(row):
    official_title = row['official_title']
    movie_correct_answer = row['year']

    # Determining difficulty level based on release year and votes
    year = int(row['year'])
    votes = int(row['votes'])
    difficulty_level = determine_difficulty_level(year, votes)

    # Getting all unique years excluding the correct answer
    all_years = list(set(movie_data['year'].unique()))
    all_years.remove(movie_correct_answer)  # Removing the correct answer

    # Selecting 3 more random incorrect options
    movie_options = [movie_correct_answer] + random.sample(all_years, 3)

    # Shuffling the options to randomizing their order
    random.shuffle(movie_options)

    # Mapping options to letters (A, B, C, D)
    options_mapping = {chr(ord('A') + i): option for i, option in enumerate(movie_options)}

    # Constructing the question dictionary
    question_dict = {
        'question': f'When was the movie {official_title} released?',
        'options': options_mapping,
        'correct_answer': chr(ord('A') + movie_options.index(movie_correct_answer)),
        'difficulty_level': difficulty_level
    }

    return question_dict

# Update quiz_game to remove the difficulty_level argument
def quiz_game():
    difficulty_levels = ['easy', 'medium', 'hard']

    # Getting user input for difficulty level
    user_difficulty = input("Choosing a difficulty level (easy, medium, hard): ").lower()

    # Validating user input
    while user_difficulty not in difficulty_levels:
        print("Invalid difficulty level. Please choose from: easy, medium, hard")
        user_difficulty = input("Choosing a difficulty level (easy, medium, hard): ").lower()

# Generating a random row index
while True:
    row_index = random.randint(0, len(movie_data) - 1)
    row = movie_data.iloc[row_index]

    # Check if the movie has a known difficulty level
    year = int(row['year'])
    votes = int(row['votes'])
    difficulty_level = determine_difficulty_level(year, votes)

    if difficulty_level in difficulty_levels:  # Check if the difficulty level is valid
        break

    # Generating the question
    question_info = generate_movie_question(row)

    # Printing the question information
    print(question_info['question'])
    
    # Printing options with letters (A, B, C, D)
    for letter, option in question_info['options'].items():
        print(f"{letter}. {option}")

    # Getting user's choice
    user_choice = input("Entering your choice (A, B, C, D): ").upper()

    # Checking if the user's choice is correct
    is_correct = user_choice == question_info['correct_answer']

    # Calculating and displaying the score
    score = calculate_score(difficulty_level, is_correct)
    
    # Providing feedback on the answer
    if is_correct:
        print("Correct!")
    else:
        print(f"Wrong! The correct answer is: {question_info['correct_answer']}")

    print(f"Your score: {score}")

# Calling the quiz_game function to play the game
quiz_game()


KeyError: 'votes'

In [ ]:
print(rating_data)

In [ ]:
twins_row = rating_data[rating_data['official_title'] == 'Frailty']
twins_row

## Closing the connection:

In [ ]:
imdb_data.close_connection()